In [5]:
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-darwin-amd64.zip
!unzip ngrok-v3-stable-darwin-amd64.zip

--2023-10-21 16:57:31--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-darwin-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 54.237.133.81, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9393876 (9.0M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-darwin-amd64.zip’

ngrok-v3-stable-dar 100%[===================>]   8.96M  12.7MB/s    in 0.7s    

2023-10-21 16:57:32 (12.7 MB/s) - ‘ngrok-v3-stable-darwin-amd64.zip’ saved [9393876/9393876]

Archive:  ngrok-v3-stable-darwin-amd64.zip
  inflating: ngrok                   


In [6]:
!./ngrok config add-authtoken 1tHaGIL1YdMNniMQKsAzqSgsiUb_7AgZfgwYUTdCzFGva4KE8

Authtoken saved to configuration file: /Users/michaelking/.ngrok2/ngrok.yml


In [1]:
import threading
from pyngrok import ngrok
from webapp2048.WebApp import socketio, app, set_public_url, call_add_step


# Function to run the Flask app with SocketIO
def run():
    socketio.run(app, host='0.0.0.0', port=5001, debug=True, use_reloader=False)

def run_async():
    # Start the Flask app in a separate thread
    threading.Thread(target=run).start()

    # Set up ngrok tunnel
    public_url = ngrok.connect(5001).public_url
    set_public_url(public_url)
    print('Public URL:', public_url)


In [2]:
run_async()

(40890) wsgi starting up on http://0.0.0.0:5001
t=2023-10-22T17:18:28-0500 lvl=warn msg="ngrok config file found at legacy location, move to XDG location" xdg_path="/Users/michaelking/Library/Application Support/ngrok/ngrok.yml" legacy_path=/Users/michaelking/.ngrok2/ngrok.yml


Public URL: https://546a-173-20-233-125.ngrok-free.app


In [15]:
import numpy as np

call_add_step({
    'epoch_number': 0,
    'game_number': 0,
    'action': 0,
    'next_board': np.array([[0, 2, 4, 2],
       [64, 32, 2, 16],
       [0, 4, 2, 8],
       [0, 4, 8, 0]]).tolist()
})

<Response [200]>


(40890) accepted ('127.0.0.1', 57066)
173.20.233.125,127.0.0.1 - - [22/Oct/2023 18:26:54] "POST /add_step HTTP/1.1" 200 166 0.000728


In [1]:
from Game2048Env import Game2048Env
from Policy2048 import create_non_masked_2048_policy_model, create_2048_policy_model

env = Game2048Env()

masked_model = create_2048_policy_model()
non_masked_model = create_non_masked_2048_policy_model()

2023-10-29 17:49:00.936737: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
state = env.reset()
mask = env.legal_actions_mask(state)

In [24]:
masked_model([state.reshape(1,4,4,1), mask.reshape(1,4)], training=False)

[<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.00599287]], dtype=float32)>,
 <tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0.22769774, 0.27601832, 0.2388441 , 0.25743982]], dtype=float32)>]

In [26]:
%%timeit

non_masked_model([state.reshape(1,4,4,1)])

3.57 ms ± 231 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
print(state)

[[[[0]
   [0]
   [0]
   [0]]

  [[2]
   [0]
   [2]
   [0]]

  [[0]
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   [0]]]]


In [9]:
from Game2048Env import Game2048Env
import numpy as np

state = [
    [0,8,0,0],
    [8,16,8,0],
    [8,0,8,0],
    [0,0,0,0],
]

Game2048Env.compute_static_reward(np.array(state))

257.1621781474655

In [42]:
import numpy as np

def log2_board(board):
    return np.log2(board, out=np.zeros_like(board, dtype='float32'), where=(board!=0))

In [43]:
log2_board(state)

/var/folders/4l/swy1ys696hb2z4jb8jxqmcqr0000gn/T/ipykernel_51265/612652193.py:4: RuntimeWarning: divide by zero encountered in log2
  return np.log2(board, out=np.zeros_like(board, dtype='float32'), where=(board!=0))


array([[  2., -inf,   1., -inf],
       [-inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf],
       [-inf, -inf, -inf, -inf]], dtype=float32)

In [39]:
np.zeros_like(state, dtype='float64')

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [2]:
from OptimizedGame2048Env import OptimizedGame2048Env
from Game2048Env import Game2048Env
import numpy as np

def test_initialization():
    env1 = Game2048Env()
    env2 = OptimizedGame2048Env()


def test_step_functionality():
    env1 = Game2048Env()
    env2 = OptimizedGame2048Env()

    env2.board = env1.board.copy()

    actions = [0, 1, 2, 3]
    for action in actions:
        obs1 = env1.half_step(env1.board, action)
        obs2 = env2.half_step(env1.board, action)
        print(obs1)
        print(obs2)
        assert np.array_equal(obs1, obs2), f"Step test failed for action {action}: Observations are not equal."
    print("Step functionality test passed!")

def test_legal_actions():
    env1 = Game2048Env()
    env2 = OptimizedGame2048Env()

    env2.board = env1.board.copy()

    legal_actions1 = env1.legal_actions_mask(env1.board)
    legal_actions2 = env2.legal_actions_mask(env2.board)
    assert np.array_equal(legal_actions1, legal_actions2), "Legal actions test failed: Legal actions are not equal."
    print("Legal actions test passed!")

def test_reset_functionality():
    env1 = Game2048Env()
    env2 = OptimizedGame2048Env()

    env2.board = env1.board.copy()

    obs1 = env1.reset()
    obs2 = env2.reset()
    assert np.array_equal(obs1, obs2), "Reset test failed: Observations are not equal."
    print("Reset functionality test passed!")

def test_rewards():
    env1 = Game2048Env()
    env2 = OptimizedGame2048Env()

    env2.board = env1.board.copy()

    actions = [0, 1, 2, 3]
    for action in actions:
        _, reward1, _, _ = env1.step(action)
        _, reward2, _, _ = env2.step(action)
        assert reward1 == reward2, f"Rewards test failed for action {action}: Rewards are not equal."
    print("Rewards test passed!")

# Run the tests
test_initialization()
test_step_functionality()
test_legal_actions()
test_reset_functionality()
test_rewards()


AttributeError: 'OptimizedGame2048Env' object has no attribute 'half_step'

In [4]:
env1 = Game2048Env()

In [1]:
from OptimizedGame2048Env import OptimizedGame2048Env
from Game2048Env import Game2048Env
import numpy as np

class TestOptimizedGame2048Env:
    def __init__(self):
        states_list = [
            np.array([[0,2,0,2],[0,2,4,8],[0,0,4,2],[0,2,4,8]]),
            np.array([[0,0,0,2],[0,2,4,8],[0,0,4,2],[0,2,4,8]]),
            np.array([[0,2,4,8],[2,4,8,16],[4,8,16,32],[8,16,32,64]])
        ]
        # Convert the list of states into a 4-dimensional array with a shape of (num_envs, 4, 4, 5)
        self.states = np.array([Game2048Env.make_observation(state, False) for state in states_list])
        self.num_envs = len(self.states)
        self.optimized_env = OptimizedGame2048Env(self.num_envs)
        self.env = Game2048Env()

    def test_board_from_state(self):
        optimized_results = OptimizedGame2048Env.board_from_state(self.states)
        results = [Game2048Env.board_from_state(state) for state in self.states]

        for i, (opt_res, res) in enumerate(zip(optimized_results, results)):
            print(f"State {i} Optimized Result:\n{opt_res}")
            print(f"State {i} Expected Result:\n{res}\n{'-'*40}")

        np.testing.assert_array_equal(optimized_results, results)


    def test_is_action_legal(self):
        for state in self.states:
            for action in range(4):
                optimized_result = OptimizedGame2048Env.is_action_legal(state, action)
                result = Game2048Env.is_action_legal(state, action)
                np.testing.assert_array_equal(optimized_result, result)

    def test_half_step(self):
        for state in self.states:
            for action in range(4):
                optimized_result = OptimizedGame2048Env.half_step(state, action)
                result = Game2048Env.half_step(state, action)
                np.testing.assert_array_equal(optimized_result, result)

    def test_compute_static_reward(self):
        optimized_results = OptimizedGame2048Env.compute_static_reward(self.states)
        results = [Game2048Env.compute_static_reward(state) for state in self.states]
        np.testing.assert_array_equal(optimized_results, results)

    def test_make_observation(self):
        for state in self.states:
            optimized_result = OptimizedGame2048Env.make_observation(state, False)
            result = Game2048Env.make_observation(state, False)
            np.testing.assert_array_equal(optimized_result, result)

    def test_move_left(self):
        optimized_results = OptimizedGame2048Env.move_left(self.states)
        results = [Game2048Env.move_left(state) for state in self.states]
        np.testing.assert_array_equal(optimized_results, results)

    def test_move_right(self):
        optimized_results = OptimizedGame2048Env.move_right(self.states)
        results = [Game2048Env.move_right(state) for state in self.states]
        np.testing.assert_array_equal(optimized_results, results)

    def test_move_up(self):
        optimized_results = OptimizedGame2048Env.move_up(self.states)
        results = [Game2048Env.move_up(state) for state in self.states]
        np.testing.assert_array_equal(optimized_results, results)

    def test_move_down(self):
        optimized_results = OptimizedGame2048Env.move_down(self.states)
        results = [Game2048Env.move_down(state) for state in self.states]
        np.testing.assert_array_equal(optimized_results, results)

    def test_legal_actions_mask_from_board(self):
        optimized_results = OptimizedGame2048Env.legal_actions_mask_from_board(self.states)
        results = [Game2048Env.legal_actions_mask_from_board(state) for state in self.states]
        np.testing.assert_array_equal(optimized_results, results)

# Running the tests
tests = TestOptimizedGame2048Env()
tests.test_board_from_state()
tests.test_is_action_legal()
tests.test_half_step()
tests.test_compute_static_reward()
tests.test_make_observation()
tests.test_move_left()
tests.test_move_right()
tests.test_move_up()
tests.test_move_down()
tests.test_legal_actions_mask_from_board()



State 0 Optimized Result:
[[0 2 0 2]
 [0 2 4 8]
 [0 0 4 2]
 [0 2 4 8]]
State 0 Expected Result:
[[0 2 0 2]
 [0 2 4 8]
 [0 0 4 2]
 [0 2 4 8]]
----------------------------------------
State 1 Optimized Result:
[[0 0 0 2]
 [0 2 4 8]
 [0 0 4 2]
 [0 2 4 8]]
State 1 Expected Result:
[[0 0 0 2]
 [0 2 4 8]
 [0 0 4 2]
 [0 2 4 8]]
----------------------------------------
State 2 Optimized Result:
[[ 0  2  4  8]
 [ 2  4  8 16]
 [ 4  8 16 32]
 [ 8 16 32 64]]
State 2 Expected Result:
[[ 0  2  4  8]
 [ 2  4  8 16]
 [ 4  8 16 32]
 [ 8 16 32 64]]
----------------------------------------


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [8]:
np.array([[0, 2, 0, 2],
 [0, 2, 4, 8],
 [0, 0, 4, 2],
 [0, 2, 4, 8]])/2

array([[0., 1., 0., 1.],
       [0., 1., 2., 4.],
       [0., 0., 2., 1.],
       [0., 1., 2., 4.]])

In [1]:
import numpy as np

board = np.array([[0, 2, 0, 2],
 [0, 2, 4, 8],
 [0, 0, 4, 2],
 [0, 2, 4, 8]])

from Game2048Env import Game2048Env

zeros = np.array([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]])
start = np.array([[2, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]])

print(Game2048Env.make_transpositions(zeros).shape)
print(Game2048Env.make_transpositions(start).shape)

(4, 4, 16)
(4, 4, 16)


In [1]:
from Game2048Env import Game2048Env

len(Game2048Env.make_transpositions([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]))

HI
transpositions=[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]]), array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]]), array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]]), array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]]), array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]]), array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]]), array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]]), array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]]), array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]]), array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]]), array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],


array([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]],
      dtype=uint

In [7]:
a = [1]
a = a[1:] + [2]

In [8]:
a

[2]

In [1]:
from Game2048Env import Game2048Env
env = Game2048Env()

In [2]:
env.board * 2

array([[0, 0, 0, 0],
       [0, 8, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 4]], dtype=uint32)

In [3]:
from Game2048Env import MAX_TILE
import numpy as np

env.reset()
board = env.board

max_board_tile = np.max(board)
transpositions = []

downward_board = np.array(board.copy())
downward_max_board_tile = max_board_tile

updward_board = np.array(board.copy())
updward_max_board_tile = max(max_board_tile, 2)

while downward_max_board_tile > 2:
    downward_board = env.transpose_values_down(downward_board)
    transpositions.insert(0, downward_board)
    downward_max_board_tile /= 2

transpositions.append(board.copy())
print(updward_board)

while updward_max_board_tile < MAX_TILE:
    updward_board = env.transpose_values_up(updward_board)
    transpositions.append(updward_board)
    updward_max_board_tile *= 2

res = np.array(transpositions, dtype=np.uint32).T.reshape((4,4, len(transpositions)))

[[0 0 0 2]
 [0 2 0 0]
 [0 0 0 0]
 [0 0 0 0]]


In [5]:
res

array([[[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0]],

       [[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0],
        [    2,     4,     8,    16,    32,    64,   128,   256,   512,
          1024,  2048,  4096,  8192, 16384, 32768, 65536],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
         